Create pull requests database

In [1]:
pip install pyvis

In [2]:
import pandas as pd
df = pd.read_csv("final_merged_discussions_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4231 entries, 0 to 4230
Data columns (total 13 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   discussion_title                       4224 non-null   object 
 1   discussion_thread_id                   4157 non-null   object 
 2   discussion_thread_created_at           4154 non-null   object 
 3   discussion_thread_author_id            4154 non-null   object 
 4   discussion_thread_author_affiliation   3172 non-null   object 
 5   discussion_thread_comment_count        4154 non-null   float64
 6   discussion_answered_or_unanswered      4154 non-null   object 
 7   discussion_comment_id                  3693 non-null   object 
 8   discussion_parent_comment_id           2204 non-null   object 
 9   discussion_comment_created_at          2364 non-null   object 
 10  discussion_comment_author              3693 non-null   object 
 11  disc

In [3]:
df.head(10)

,discussion_title,discussion_thread_id,discussion_thread_created_at,discussion_thread_author_id,discussion_thread_author_affiliation,discussion_thread_comment_count,discussion_answered_or_unanswered,discussion_comment_id,discussion_parent_comment_id,discussion_comment_created_at,discussion_comment_author,discussion_comment_author_affiliation,discussion_comment_text
0,Localization Architecture Design,D_kwDOAnasMc4AP640,2021-12-22T10:22:48Z,mitsudome-r,autoware-developers-tier4,4.0,Answered,DC_kwDOAnasMc4AHFoI,NaN,2021-12-22 11:58:10+00:00,mitsudome-r,autoware-developers-tier4,Here's the meeting minutes from the discussion...
1,Localization Architecture Design,D_kwDOAnasMc4AP640,2021-12-22T10:22:48Z,mitsudome-r,autoware-developers-awf,4.0,Answered,DC_kwDOAnasMc4AIFQu,DC_kwDOAnasMc4AHFoI,2022-02-06 03:58:37+00:00,xczhanjun,autoware-developers-awf,How about add an end 2 end AI module together ...
2,Localization Architecture Design,D_kwDOAnasMc4AP640,2021-12-22T10:22:48Z,mitsudome-r,autoware-maintainers,4.0,Answered,DC_kwDOAnasMc4AI3TZ,DC_kwDOAnasMc4AHFoI,2022-03-09 12:15:54+00:00,xmfcx,autoware-maintainers,@xczhanjun As long as it can replicate the inp...
3,Localization Architecture Design,D_kwDOAnasMc4AP640,2021-12-22T10:22:48Z,mitsudome-r,autoware-admins,4.0,Answered,DC_kwDOAnasMc4AIdYt,NaN,2022-02-21 01:57:03+00:00,maxime-clem,autoware-admins,I would like to propose an additional feature:...
4,Localization Architecture Design,D_kwDOAnasMc4AP640,2021-12-22T10:22:48Z,mitsudome-r,autoware-developers-udel,4.0,Answered,DC_kwDOAnasMc4AIdts,DC_kwDOAnasMc4AIdYt,2022-02-21 08:19:37+00:00,mitsudome-r,autoware-developers-udel,@YamatoAndo Do you have any opinion on this?
5,Localization Architecture Design,D_kwDOAnasMc4AP640,2021-12-22T10:22:48Z,mitsudome-r,autoware-developers-upenn,4.0,Answered,DC_kwDOAnasMc4AId00,DC_kwDOAnasMc4AIdYt,2022-02-21 09:29:11+00:00,YamatoAndo,autoware-developers-upenn,"I agree with this propose.\r\nCurrently, the a..."
6,Localization Architecture Design,D_kwDOAnasMc4AP640,2021-12-22T10:22:48Z,mitsudome-r,autoware-developers-tier4,4.0,Answered,DC_kwDOAnasMc4AIeE2,DC_kwDOAnasMc4AIdYt,2022-02-21 11:34:46+00:00,brkay54,baidu usa,I agree with @YamatoAndo but also we should ch...
7,Localization Architecture Design,D_kwDOAnasMc4AP640,2021-12-22T10:22:48Z,mitsudome-r,autoware-developers-awf,4.0,Answered,DC_kwDOAnasMc4AIjRz,NaN,2022-02-24 06:29:56+00:00,YamatoAndo,autoware-developers-awf,"In the current architecture, the localization ..."
8,Localization Architecture Design,D_kwDOAnasMc4AP640,2021-12-22T10:22:48Z,mitsudome-r,autoware-maintainers,4.0,Answered,DC_kwDOAnasMc4AIlHy,DC_kwDOAnasMc4AIjRz,2022-02-25 05:46:28+00:00,maxime-clem,autoware-developers-leodrive,I like both options 1 and 2. I think if few no...
9,Localization Architecture Design,D_kwDOAnasMc4AP640,2021-12-22T10:22:48Z,mitsudome-r,autoware-admins,4.0,Answered,DC_kwDOAnasMc4AIldt,DC_kwDOAnasMc4AIjRz,2022-02-25 10:30:26+00:00,YamatoAndo,autoware-maintainers,Option 3 seems to be popular in Tier IV.


Code 2: Varied sizes of the spheres

In [4]:
pip install plotly

In [5]:
pip install networkx

In [6]:
import pandas as pd
import numpy as np
import plotly.express as px

# Read the CSV file
df = pd.read_csv("final_merged_discussions_data.csv")

# Drop rows where affiliation info is missing
df = df.dropna(subset=['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation'])

# Create 'comment_or_reply' column
df['comment_or_reply'] = df.apply(lambda row: 'comment' if row['discussion_thread_comment_count'] > 0 and pd.isna(row['discussion_parent_comment_id']) else 'reply', axis=1)

# Additional calculations for inside/outside consumption and contribution
df['inside_consumption'] = 0
df['outside_consumption'] = 0
df['inside_contribution'] = 0
df['outside_contribution'] = 0

In [7]:
### FINDING THE CONTRIBUTER-CONSUMER RELATIONSHIPS ###

# Create pairs for the first scenario: thread creators and comment authors
pairs_scenario_1 = df[df['comment_or_reply'].isin(['comment', 'reply'])][['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation', 'discussion_thread_id']].values

# Create pairs for the second scenario: comment authors (comment) and comment authors (reply) within the same thread
pairs_scenario_2_consumer = df[df['comment_or_reply'] == 'comment'][['discussion_thread_id', 'discussion_comment_author_affiliation']].values
pairs_scenario_2_contributor = df[df['comment_or_reply'] == 'reply'][['discussion_thread_id', 'discussion_comment_author_affiliation']].values

# Ensure both scenarios have the same number of columns
pairs_scenario_2_consumer = np.column_stack((pairs_scenario_2_consumer, np.empty(len(pairs_scenario_2_consumer))))
pairs_scenario_2_contributor = np.column_stack((pairs_scenario_2_contributor, np.empty(len(pairs_scenario_2_contributor))))

# Concatenate pairs from both scenarios
pairs = np.concatenate([pairs_scenario_1, pairs_scenario_2_consumer, pairs_scenario_2_contributor])

# Filter unique thread-consumer pairs using Pandas
# Convert relevant columns to strings before finding unique pairs
pairs_str = pairs[:, [0, 2, 1]].astype(str)
unique_thread_consumer_pairs = np.unique(pairs_str, axis=0)

# Convert to DataFrame
unique_thread_consumer_pairs_df = pd.DataFrame(unique_thread_consumer_pairs, columns=['discussion_thread_author_affiliation', 'discussion_thread_id', 'discussion_comment_author_affiliation'])
unique_thread_consumer_pairs_df.drop_duplicates(inplace=True)

# Convert back to NumPy array
unique_thread_consumer_pairs = unique_thread_consumer_pairs_df.values

# Filter unique thread-contributor pairs
# Convert relevant columns to strings before finding unique pairs
pairs_str_contributor = pairs[:, [0, 1, 2]].astype(str)
unique_thread_contributor_pairs = np.unique(pairs_str_contributor, axis=0)

# Concatenate both unique pairs
unique_pairs = np.concatenate([unique_thread_consumer_pairs, unique_thread_contributor_pairs])

# Calculate 'threads' for each user
threads_count = df['discussion_thread_author_id'].value_counts()
df['threads'] = df['discussion_thread_author_id'].map(threads_count).fillna(0)

# Initialize 'contribution_internal' and 'contribution_external' columns
df['contribution_internal'] = 0
df['contribution_external'] = 0

In [8]:
# Iterate through each unique thread id
for thread_id in df['discussion_thread_id'].unique():
    # Get unique comment authors for the thread
    comment_authors = df.loc[df['discussion_thread_id'] == thread_id, 'discussion_comment_author'].unique()

    # Iterate through unique comment authors for the thread
    for comment_author in comment_authors:
        # Filter rows based on thread_id and comment_author
        filter_condition = (df['discussion_thread_id'] == thread_id) & (df['discussion_comment_author'] == comment_author)
        matching_rows = df[filter_condition]

        # Check if there are any matching rows
        if not matching_rows.empty:
            # Check if the affiliations match for thread creator and comment author
            if matching_rows['discussion_thread_author_affiliation'].iloc[0] == matching_rows['discussion_comment_author_affiliation'].iloc[0]:
                # Increment contribution_internal value for the comment author
                df.loc[filter_condition, 'contribution_internal'] += 1
            else:
                # Increment contribution_external value for the comment author
                df.loc[filter_condition, 'contribution_external'] += 1

# Iterate through each unique thread id to calculate 'contribution_external'
for thread_id in df['discussion_thread_id'].unique():
    # Get unique comment authors (excluding thread creator) for that thread
    comment_authors = df.loc[df['discussion_thread_id'] == thread_id, 'discussion_comment_author'].values
    thread_creator_rows = df.loc[df['discussion_thread_id'] == thread_id, 'discussion_thread_author_id']
    if not thread_creator_rows.empty:
        thread_creator = thread_creator_rows.iloc[0]
        comment_authors = [author for author in comment_authors if author != thread_creator]
        # Increment 'contribution_external' for each unique comment author
        df.loc[df['discussion_thread_id'] == thread_id, 'contribution_external'] += len(set(comment_authors))

In [9]:
### INSIDE-OUTSIDE CONTRIBUTION/CONSUMPTION LOGIC ###

# Additional calculations for inside/outside opportunity
df['inside_opportunity'] = 0
df['outside_opportunity'] = 0

# Iterate through pairs and apply the inside-outside contribution/consumption/opportunity logic
for pair in unique_pairs:
    consumer_affiliation, contributor_affiliation, thread_id = pair

    if consumer_affiliation != contributor_affiliation:
        # Different affiliations, increase outside contribution of contributor, outside consumption of consumer
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_thread_author_affiliation'] == contributor_affiliation) & (df['discussion_thread_id'] == thread_id), 'outside_contribution'] += 1
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_id'] == thread_id), 'outside_consumption'] += 1
        # Increase outside opportunity
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_author_affiliation'] == contributor_affiliation), 'outside_opportunity'] += 1
    else:
        # Same affiliation, increase inside contribution and inside consumption
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_thread_author_affiliation'] == contributor_affiliation) & (df['discussion_thread_id'] == thread_id), 'inside_contribution'] += 1
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_id'] == thread_id), 'inside_consumption'] += 1
        # Increase inside opportunity
        df.loc[(df['comment_or_reply'].isin(['reply', 'comment'])) & (df['discussion_comment_author_affiliation'] == consumer_affiliation) & (df['discussion_thread_author_affiliation'] == contributor_affiliation), 'inside_opportunity'] += 1

# Calculate collaboration index
df['collaboration_index'] = (
    (df['inside_contribution'] / df['inside_opportunity']) /
    (df['outside_contribution'] / df['outside_opportunity'])
)

# Save collaboration indexes, threads, contribution_internal, contribution_external to a CSV file
result_df = df[['discussion_comment_author_affiliation', 'discussion_thread_author_affiliation', 'collaboration_index', 'threads', 'contribution_internal', 'contribution_external']]
result_df.to_csv("collaboration_and_threads.csv", index=False)

In [10]:
### INSIDE-OUTSIDE CONTRIBUTION/CONSUMPTION: VISUALIZATION ###

# Group by user and sum the counts for both 'contribution_internal' and 'contribution_external'
user_stats_contribution = df.groupby('discussion_comment_author').agg({
    'contribution_internal': 'sum',
    'contribution_external': 'sum'
}).reset_index()

# Visualizations for user stats
fig_contribution_internal = px.bar(user_stats_contribution, x='discussion_comment_author', y='contribution_internal',
                                   labels={'index': 'User', 'value': 'Count'},
                                   title='Contribution Internal for Each User')

fig_contribution_external = px.bar(user_stats_contribution, x='discussion_comment_author', y='contribution_external',
                                   labels={'index': 'User', 'value': 'Count'},
                                   title='Contribution External for Each User')

# Save the plots as interactive HTML files
fig_contribution_internal.write_html("contribution_internal_user.html")
fig_contribution_external.write_html("contribution_external_user.html")

In [11]:
# Initialize 'contribution_internal_org' and 'contribution_external_org' columns for each unique affiliation
df['contribution_internal_org'] = 0
df['contribution_external_org'] = 0

# Iterate through each unique thread id
for thread_id in df['discussion_thread_id'].unique():
    # Get unique comment authors for the thread
    comment_authors = df.loc[df['discussion_thread_id'] == thread_id, 'discussion_comment_author'].unique()

    # Iterate through unique comment authors for the thread
    for comment_author in comment_authors:
        # Filter rows based on thread_id and comment_author
        filter_condition = (df['discussion_thread_id'] == thread_id) & (df['discussion_comment_author'] == comment_author)
        matching_rows = df[filter_condition]

        # Check if there are any matching rows
        if not matching_rows.empty:
            # Check if the affiliations match for thread creator and comment author
            thread_author_affiliation = matching_rows['discussion_thread_author_affiliation'].iloc[0]
            comment_author_affiliation = matching_rows['discussion_comment_author_affiliation'].iloc[0]

            # Increment contribution_internal_org and contribution_external_org based on affiliation
            if thread_author_affiliation == comment_author_affiliation:
                # Increment contribution_internal_org for the comment author's affiliation
                df.loc[filter_condition, 'contribution_internal_org'] += 1
            else:
                # Increment contribution_external_org for the comment author's affiliation
                df.loc[filter_condition, 'contribution_external_org'] += 1

In [12]:
# Group by affiliation and sum the counts for both 'contribution_internal_org' and 'contribution_external_org'
org_stats = df.groupby('discussion_comment_author_affiliation').agg({
    'contribution_internal_org': 'sum',
    'contribution_external_org': 'sum'
}).reset_index()

# Visualizations for organization stats
fig_contribution_internal_org = px.bar(org_stats, x='discussion_comment_author_affiliation', y='contribution_internal_org',
                                       labels={'index': 'Affiliation', 'value': 'Count'},
                                       title='Contribution Internal for Each Organization')

fig_contribution_external_org = px.bar(org_stats, x='discussion_comment_author_affiliation', y='contribution_external_org',
                                       labels={'index': 'Affiliation', 'value': 'Count'},
                                       title='Contribution External for Each Organization')

# Save the plots as interactive HTML files
fig_contribution_internal_org.write_html("contribution_internal_org.html")
fig_contribution_external_org.write_html("contribution_external_org.html")

In [13]:
# Initialize 'thread_user' and 'thread_org' columns
df['thread_user'] = 0
df['thread_org'] = 0

# Get unique users and affiliations
unique_users = df['discussion_comment_author'].unique()
unique_affiliations = df['discussion_comment_author_affiliation'].unique()

# Calculate 'thread_user' for each user
for user in unique_users:
    user_thread_count = df[df['discussion_comment_author'] == user]['discussion_thread_id'].nunique()
    df.loc[df['discussion_comment_author'] == user, 'thread_user'] = user_thread_count

# Calculate 'thread_org' for each affiliation
for affiliation in unique_affiliations:
    affiliation_thread_count = df[df['discussion_comment_author_affiliation'] == affiliation]['discussion_thread_id'].nunique()
    df.loc[df['discussion_comment_author_affiliation'] == affiliation, 'thread_org'] = affiliation_thread_count

# Save collaboration indexes, threads, thread_internal, thread_external, thread_user, thread_org to a CSV file
result_df = df[['discussion_comment_author_affiliation', 'discussion_thread_author_affiliation', 'collaboration_index', 'threads', 'contribution_internal', 'contribution_external', 'thread_user', 'thread_org']]
result_df.to_csv("collaboration_and_threads.csv", index=False)

In [14]:
### Create thread_user.html and thread_org.html figures ###

# Group by user and affiliation and sum the counts for 'thread_user' and 'thread_org'
user_stats = df.groupby('discussion_comment_author').agg({
    'thread_user': 'max'
}).reset_index()

org_stats = df.groupby('discussion_comment_author_affiliation').agg({
    'thread_org': 'max'
}).reset_index()

# Visualizations for user and affiliation stats
fig_thread_user = px.bar(user_stats, x='discussion_comment_author', y='thread_user',
                          labels={'index': 'User', 'value': 'Count'},
                          title='Thread User for Each User')

fig_thread_org = px.bar(org_stats, x='discussion_comment_author_affiliation', y='thread_org',
                         labels={'index': 'Affiliation', 'value': 'Count'},
                         title='Thread Org for Each Affiliation')

# Save the plots as interactive HTML files
fig_thread_user.write_html("thread_user.html")
fig_thread_org.write_html("thread_org.html")

In [15]:
import pandas as pd
import numpy as np
import plotly.express as px

# Read the CSV file
df = pd.read_csv("final_merged_discussions_data.csv")

# Drop rows where affiliation info is missing
df = df.dropna(subset=['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation'])

# Create 'comment_or_reply' column
df['comment_or_reply'] = df.apply(lambda row: 'comment' if row['discussion_thread_comment_count'] > 0 and pd.isna(row['discussion_parent_comment_id']) else 'reply', axis=1)

# Additional calculations for inside/outside consumption and contribution
df['inside_consumption'] = 0
df['outside_consumption'] = 0
df['inside_contribution'] = 0
df['outside_contribution'] = 0

### FINDING THE CONTRIBUTER-CONSUMER RELATIONSHIPS ###

# Create pairs for the first scenario: thread creators and comment authors
pairs_scenario_1 = df[df['comment_or_reply'].isin(['comment', 'reply'])][['discussion_thread_author_affiliation', 'discussion_comment_author_affiliation', 'discussion_thread_id']].values

# Create pairs for the second scenario: comment authors (comment) and comment authors (reply) within the same thread
pairs_scenario_2_consumer = df[df['comment_or_reply'] == 'comment'][['discussion_thread_id', 'discussion_comment_author_affiliation']].values
pairs_scenario_2_contributor = df[df['comment_or_reply'] == 'reply'][['discussion_thread_id', 'discussion_comment_author_affiliation']].values

# Ensure both scenarios have the same number of columns
pairs_scenario_2_consumer = np.column_stack((pairs_scenario_2_consumer, np.empty(len(pairs_scenario_2_consumer))))
pairs_scenario_2_contributor = np.column_stack((pairs_scenario_2_contributor, np.empty(len(pairs_scenario_2_contributor))))

# Concatenate pairs from both scenarios
pairs = np.concatenate([pairs_scenario_1, pairs_scenario_2_consumer, pairs_scenario_2_contributor])

# Filter unique thread-consumer pairs using Pandas
# Convert relevant columns to strings before finding unique pairs
pairs_str = pairs[:, [0, 2, 1]].astype(str)
unique_thread_consumer_pairs = np.unique(pairs_str, axis=0)

# Convert to DataFrame
unique_thread_consumer_pairs_df = pd.DataFrame(unique_thread_consumer_pairs, columns=['discussion_thread_author_affiliation', 'discussion_thread_id', 'discussion_comment_author_affiliation'])
unique_thread_consumer_pairs_df.drop_duplicates(inplace=True)

# Convert back to NumPy array
unique_thread_consumer_pairs = unique_thread_consumer_pairs_df.values

# Filter unique thread-contributor pairs
# Convert relevant columns to strings before finding unique pairs
pairs_str_contributor = pairs[:, [0, 1, 2]].astype(str)
unique_thread_contributor_pairs = np.unique(pairs_str_contributor, axis=0)

# Concatenate both unique pairs
unique_pairs = np.concatenate([unique_thread_consumer_pairs, unique_thread_contributor_pairs])

# Calculate 'threads' for each user
threads_count = df['discussion_thread_author_id'].value_counts()
df['threads'] = df['discussion_thread_author_id'].map(threads_count).fillna(0)

In [16]:
# Initialize 'contribution_internal' and 'contribution_external' columns
df['contribution_internal'] = 0
df['contribution_external'] = 0

# Initialize 'thread_user', 'thread_internal', 'thread_external', 'thread_org', 'thread_internal_org', 'thread_external_org' columns
df['thread_user'] = 0
df['thread_internal'] = 0
df['thread_external'] = 0
df['thread_org'] = 0
df['thread_internal_org'] = 0
df['thread_external_org'] = 0

In [17]:
# Get unique users and affiliations
unique_users = df['discussion_comment_author'].unique()
unique_affiliations = df['discussion_comment_author_affiliation'].unique()

In [18]:
# Calculate 'thread_user', 'thread_internal', 'thread_external', 'thread_org', 'thread_internal_org', 'thread_external_org' for each user
for user in unique_users:
    user_thread_count = df[df['discussion_comment_author'] == user]['discussion_thread_id'].nunique()
    df.loc[df['discussion_comment_author'] == user, 'thread_user'] = user_thread_count

    # Calculate 'thread_internal', 'thread_external', 'thread_internal_org', 'thread_external_org' for each user with respect to 'discussion_thread_id'
    for thread_id in df[df['discussion_comment_author'] == user]['discussion_thread_id'].unique():
        comment_authors = df.loc[df['discussion_thread_id'] == thread_id, 'discussion_comment_author'].unique()
        affiliations = df.loc[df['discussion_comment_author'] == user, 'discussion_comment_author_affiliation'].unique()

        for unique_author in comment_authors:
            unique_author_affiliation = df.loc[(df['discussion_comment_author'] == unique_author) & (df['discussion_thread_id'] == thread_id), 'discussion_comment_author_affiliation'].iloc[0]

            if not df[df['discussion_thread_id'] == thread_id]['discussion_thread_author_affiliation'].empty:
              if unique_author_affiliation == df[df['discussion_thread_id'] == thread_id]['discussion_thread_author_affiliation'].iloc[0]:
                df.loc[(df['discussion_comment_author'] == user) & (df['discussion_thread_id'] == thread_id), 'thread_internal'] += 1
                df.loc[(df['discussion_comment_author'] == user) & (df['discussion_thread_author_affiliation'] == unique_author_affiliation), 'thread_internal_org'] += 1
              else:
                df.loc[(df['discussion_comment_author'] == user) & (df['discussion_thread_id'] == thread_id), 'thread_external'] += 1
                df.loc[(df['discussion_comment_author'] == user) & (df['discussion_thread_author_affiliation'] == unique_author_affiliation), 'thread_external_org'] += 1

In [19]:
# Iterate through each unique thread id to calculate 'contribution_internal' and 'contribution_external'
for thread_id in df['discussion_thread_id'].unique():
    # Get unique comment authors for the thread
    comment_authors = df.loc[df['discussion_thread_id'] == thread_id, 'discussion_comment_author'].unique()

    # Iterate through unique comment authors for the thread
    for comment_author in comment_authors:
        # Filter rows based on thread_id and comment_author
        filter_condition = (df['discussion_thread_id'] == thread_id) & (df['discussion_comment_author'] == comment_author)
        matching_rows = df[filter_condition]

        # Check if there are any matching rows
        if not matching_rows.empty:
            # Check if the affiliations match for thread creator and comment author
            if matching_rows['discussion_thread_author_affiliation'].iloc[0] == matching_rows['discussion_comment_author_affiliation'].iloc[0]:
                # Increment contribution_internal value for the comment author
                df.loc[filter_condition, 'contribution_internal'] += 1
            else:
                # Increment contribution_external value for the comment author
                df.loc[filter_condition, 'contribution_external'] += 1

In [20]:
# Calculate collaboration index
# df['collaboration_index'] = (
#     (df['inside_contribution'] / df['inside_opportunity']) /
#     (df['outside_contribution'] / df['outside_opportunity'])
# )

# Save collaboration indexes, threads, contribution_internal, contribution_external, thread_internal, thread_external, thread_user, thread_org, thread_internal_org, thread_external_org to a CSV file
result_df = df[['discussion_comment_author_affiliation', 'discussion_thread_author_affiliation', 'threads', 'contribution_internal', 'thread_internal', 'thread_external', 'thread_user', 'thread_org', 'thread_internal_org', 'thread_external_org']]
result_df.to_csv("collaboration_and_threads.csv", index=False)

### Create thread_internal_user.html, thread_external_user.html, thread_internal_org.html, and thread_external_org.html figures ###

# Group by user and affiliation and sum the counts for 'thread_internal', 'thread_external', 'thread_internal_org', 'thread_external_org'
user_stats_internal = df.groupby('discussion_comment_author_affiliation').agg({
    'thread_internal': 'sum',
    'thread_external': 'sum',
    'thread_internal_org': 'sum',
    'thread_external_org': 'sum'
}).reset_index()

# Visualizations for user stats
fig_internal_user = px.bar(user_stats_internal, x='discussion_comment_author_affiliation', y=['thread_internal', 'thread_internal_org'],
                           labels={'index': 'Affiliation', 'value': 'Count'},
                           title='Thread Internal for Each Affiliation')

fig_external_user = px.bar(user_stats_internal, x='discussion_comment_author_affiliation', y=['thread_external', 'thread_external_org'],
                           labels={'index': 'Affiliation', 'value': 'Count'},
                           title='Thread External for Each Affiliation')

# Save the plots as interactive HTML files
fig_internal_user.write_html("thread_internal_user.html")
fig_external_user.write_html("thread_external_user.html")

In [21]:
# Group by affiliation and sum the counts for 'thread_internal_org' and 'thread_external_org'
affiliation_stats = df.groupby('discussion_thread_author_affiliation').agg({
    'thread_internal_org': 'sum',
    'thread_external_org': 'sum'
}).reset_index()

# Visualizations for affiliation stats
fig_internal_org = px.bar(affiliation_stats, x='discussion_thread_author_affiliation', y='thread_internal_org',
                         labels={'index': 'Affiliation', 'value': 'Count'},
                         title='Thread Internal for Each Affiliation')

fig_external_org = px.bar(affiliation_stats, x='discussion_thread_author_affiliation', y='thread_external_org',
                         labels={'index': 'Affiliation', 'value': 'Count'},
                         title='Thread External for Each Affiliation')

# Save the plots as interactive HTML files
fig_internal_org.write_html("thread_internal_org.html")
fig_external_org.write_html("thread_external_org.html")